# Scape Sector information for multiple Mutual fund from Yahoo Finance

This works using splinter to open the url in the browser.  We must do it this way because the data we are looking for is not it the HTML if look at it outside of a browser. The data is loaded with the webpage by a java script.  So if we just did a "requests.get(url)" the data we are after would not be there.  We must 
load a browser and scrape/pars that.

### Next Steps in Python
* it is not going to the next mutual fund
* Convert the values in the data frame from strings to values
* Export that data frame as a CSV file

### Next step in Excel 
* Import CSV file in my financial spreadsheet
* Report my exposer to each of these sectors.


In [1]:
# Website:  https://finance.yahoo.com/quote/VTSAX/holdings?p=VTSAX

# Import Splinter and BeautifulSoup
from splinter import Browser
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import os
import json

In [2]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - Downloading: 100%|██████████| 6.80M/6.80M [00:00<00:00, 21.6MB/s]


---
## Create Functions

In [3]:
# FUNCTION
# DESCRIPTION:  This function is used to splits a string into 2 parts at the first number. 
#               A dictionary is created with the first string as the key and the second string as the value
# ARGUMENTS:  A string that starts with a string and is followed by a value.
# RETURN:  Returns a dictionary where the letters in the string form the key and the numbers become the values of the dictionary entry.

def split_string_at_first_number(string):
    index = len(string)

    # count the characters until you find the first number.
    for i, char in enumerate(string):
        
        # break out of the for loop when you find a digit.
        if char.isdigit():
            index = i
            break

    # divide the string in to the key and the value based on the location of the first digit.    
    key = string[:index]
    value = string[index:]

    # Put the key and value into a dictionary and return the dictionary
    dictionary = {key: value}
    return dictionary

In [4]:
# FUNCTION
# DESCRIPTION:  Find the industry sector loading of a mutual fund by scraping a Yahoo finance webpage
# ARGUMENTS:  The mutual fund ticker symbol
# RETURN:  a data frame containing the industry sector loading of that fund

def one_ticker(mutual_fund):
      # build the URL where the data is located.
      # mutual_fund is the ticker symble of the fund to be scraped
      # Constant pieces of the Yahoo finance website to be scraped
      url_part_1 = 'https://finance.yahoo.com/quote/'
      url_part_2 = '/holdings?p='
      
      # Buile the full url to be scrapped.
      url = url_part_1 + mutual_fund + url_part_2 + mutual_fund

      # Print the full URL - this is done only for diagnostic purposes
      print(url)

      # open a browser to the required URL
      browser.visit(url)

      # Create a Beautiful Soup object
      # This will open the url in the browser.  We must do it this way because the data we are looking for is not it the HTML if look at it outside of a browser.
      # The data is loaded with the webpage by a java script.  So if we just did a "requests.get(url)" the data we are after would not be there.  We must 
      # load a browser and scrape/pars that.

      html = browser.html
      soup = BeautifulSoup(html, 'html.parser')

      # Start a dictionary with the key-value pair for the fund name.
      Industry_sec ={'Fund_Code': fund}

      # get the industry sector information from the url
      # The search strings come from "inspecting" the website where the data is located.
      # The website does not unquicly identify sector information. It get extra information that will need to be cleaned out 
      Industry_sector = soup.find_all('div', class_='Bdbw(1px) Bdbc($seperatorColor) Bdbs(s) H(25px) Pt(10px)')

      for sector in Industry_sector:

                  # return sector name and value
                  sec = str(sector.text)

                  # seperate the name and value and update the dictionary Industry_sec with the new key-value pair
                  Industry_sec.update (split_string_at_first_number(sec))

      # build a dataframe using the Industry_sec dictionary
      df = pd.DataFrame(Industry_sec, index =[0])
      #  df.head()
      return df

---
# Start the search for my information

In [5]:
# Many times the returned data frame contains more column than just the industry sectors.
# This is a list of the columns that are actually industry sectors
industry_sector_col = [
      'Fund_Code', 
      'Basic Materials', 
      'Consumer Cyclical', 
      'Financial Services',
      'Real Estate', 
      'Consumer Defensive', 
      'Healthcare', 'Utilities',
      'Communication Services', 
      'Energy', 
      'Industrials', 
      'Technology']

# Create an empty data frame with the column headings
df_rollup = pd.DataFrame(columns=industry_sector_col)

# list of mutual funds that I want to scrape.
funds = [
      'BDBKX',
      'DIA', 
      'QQQ',
      'RERGX', 
      'RWMGX',  
      'VAW', 
      'VBK', 
      'VDC', 
      'VEMPX', 
      'VEUSX', 
      'VEXAX', 
      'VFIAX', 
      'VHT', 
      'VIGIX', 
      'VIS', 
      'VMVAX', 
      'VOO', 
      'VOT', 
      'VPU', 
      'VTI', 
      'VTIAX',
      'VTPSX', 
      'VXF', 
      'VXUS', 
      'WFSPX']

In [6]:

# for loop moving through the list of funds
for fund in funds:

      # print (fund)
      # print (url)
      dfa = one_ticker(fund)
      dfa.head()

      # clean the df
      dfa = dfa[industry_sector_col]
      # print (dfa)

      # merge this data frame with the others
      
      # df_rollup = [df_rollup, dfa]

      df_rollup = pd.concat([df_rollup, dfa])


https://finance.yahoo.com/quote/BDBKX/holdings?p=BDBKX
https://finance.yahoo.com/quote/DIA/holdings?p=DIA
https://finance.yahoo.com/quote/QQQ/holdings?p=QQQ
https://finance.yahoo.com/quote/RERGX/holdings?p=RERGX
https://finance.yahoo.com/quote/RWMGX/holdings?p=RWMGX
https://finance.yahoo.com/quote/VAW/holdings?p=VAW
https://finance.yahoo.com/quote/VBK/holdings?p=VBK
https://finance.yahoo.com/quote/VDC/holdings?p=VDC
https://finance.yahoo.com/quote/VEMPX/holdings?p=VEMPX
https://finance.yahoo.com/quote/VEUSX/holdings?p=VEUSX
https://finance.yahoo.com/quote/VEXAX/holdings?p=VEXAX
https://finance.yahoo.com/quote/VFIAX/holdings?p=VFIAX
https://finance.yahoo.com/quote/VHT/holdings?p=VHT
https://finance.yahoo.com/quote/VIGIX/holdings?p=VIGIX
https://finance.yahoo.com/quote/VIS/holdings?p=VIS
https://finance.yahoo.com/quote/VMVAX/holdings?p=VMVAX
https://finance.yahoo.com/quote/VOO/holdings?p=VOO
https://finance.yahoo.com/quote/VOT/holdings?p=VOT
https://finance.yahoo.com/quote/VPU/holdings?p

In [7]:
df_rollup.head(30)

,Fund_Code,Basic Materials,Consumer Cyclical,Financial Services,Real Estate,Consumer Defensive,Healthcare,Utilities,Communication Services,Energy,Industrials,Technology
0,BDBKX,4.23%,10.27%,15.86%,7.66%,4.21%,16.08%,3.46%,2.56%,6.66%,14.71%,13.51%
0,DIA,1.08%,13.78%,20.14%,0.00%,7.08%,19.96%,0.00%,2.75%,3.22%,14.57%,17.17%
0,QQQ,0.00%,15.24%,0.66%,0.21%,5.73%,6.48%,1.15%,16.27%,0.41%,3.86%,49.99%
0,RERGX,8.12%,13.81%,11.12%,0.35%,6.18%,13.17%,0.94%,3.07%,8.21%,13.65%,14.95%
0,RWMGX,2.71%,6.96%,14.01%,1.98%,8.46%,18.38%,2.75%,5.08%,5.04%,11.17%,18.99%
0,VAW,85.65%,11.18%,0.00%,0.00%,0.00%,0.75%,0.00%,0.00%,0.52%,1.48%,0.00%
0,VBK,2.25%,11.91%,3.53%,5.93%,3.66%,19.25%,0.42%,2.93%,4.77%,16.02%,26.75%
0,VDC,0.19%,0.65%,0.00%,0.00%,97.71%,1.09%,0.00%,0.00%,0.00%,0.18%,0.00%
0,VEMPX,3.72%,11.70%,14.83%,6.79%,2.96%,12.40%,1.84%,3.73%,4.45%,14.15%,20.53%
0,VEUSX,6.67%,10.41%,17.69%,1.95%,10.80%,13.79%,3.64%,3.92%,5.97%,15.79%,7.23%


In [8]:
# close down the browser opened by ChromeDriverManager
browser.quit()

In [9]:
# Write the data frame as a CSV file

# build the path and file name for the CSV file.
file_one = os.path.join('.','Resources', 'Industry_Sector_Load.csv')

# Write out the file as the CSV file with headers but without an index.
df_rollup.to_csv(file_one, index=False, header=True)                    